# open CV

TODO: create security cameras

TODO: create a Google VR app that allows you to type on any surface using the front camera






detect people?


In [ ]:
import os
import requests
import cv2
import numpy as np
import_notebook('motion detection', globals())


# Initialize HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def get_image(image_path):

  # Load image
  if os.path.exists(image_path):
    image = cv2.imread(image_path)
  else:
    # Fetch the image from the URL
    response = requests.get(image_path)
    response.raise_for_status()  # Raise an error if the request fails

    # Convert to a NumPy array for OpenCV
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

  # Resize image (optional, improves detection speed)
  image = cv2.resize(image, (800, 600))

  return image

def trace_boxes(image_path):

  if isinstance(image_path, str):
    image = get_image(image_path)
  else:
    image = image_path

  # Convert to grayscale (not necessary, but can improve performance)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Detect people
  boxes, weights = hog.detectMultiScale(gray, winStride=(4, 4), padding=(8, 8), scale=1.05)

  return boxes

def trace_people(image_path):

  if isinstance(image_path, str):
    image = get_image(image_path)
  else:
    image = image_path

  boxes = trace_boxes(image)

  # Draw bounding boxes
  for (x, y, w, h) in boxes:
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

  return image

def main(image_path):
  prev_frame = None
  image = None
  while True:
    # Create a random image (or load a new one dynamically)
    image = get_image(image_path)

    if(prev_frame is not None):
      frame = image_grayscale(image)
      image = diff_images(prev_frame, frame)
      motion = percent_motion(image)
      # TODO: if motion is > 1.0 start recording and compile a video and upload to youtube
      print(motion)
    else:
      frame = image_grayscale(image)
      image = trace_people(image)  # Random image example

    prev_frame = frame.copy()

    # Display the image
    cv2.imshow("People Detection", image)

    # Wait for 1 second (1000 ms) and check for key press
    if cv2.waitKey(1000) & 0xFF == ord('q'):  # Press 'q' to exit
        break

  cv2.destroyAllWindows()

__all__ = {
  "main": main,
  "trace_people": trace_people
}



motion detection?


In [ ]:

import cv2
import numpy as np

def percent_motion(thresh):
  white_pixels = np.count_nonzero(thresh == 255)
  total_pixels = thresh.size

  # Calculate percentage
  white_percentage = (white_pixels / total_pixels) * 100
  return white_percentage

def image_grayscale(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return gray_image

def diff_images(prev_frame, frame):
  # Compute absolute difference between current and previous frame
  diff = cv2.absdiff(prev_frame, frame)

  # Threshold the difference image
  _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
  # Update previous frame
  # prev_frame = frame.copy()
  return thresh

__all__ = {
  "percent_motion": percent_motion,
  "image_grayscale": image_grayscale,
  "diff_images": diff_images
}



python write video?


In [ ]:
import cv2
import glob

# Set output video properties
fps = 30
output_file = "output.mp4"

def write_video():
  # Get list of images
  image_files = sorted(glob.glob("frame_*.jpg"))  # Adjust pattern as needed

  # Read first image to get dimensions
  frame = cv2.imread(image_files[0])
  height, width, _ = frame.shape

  # Define VideoWriter object
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
  out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

  # Write images to video
  for img_path in image_files:
      frame = cv2.imread(img_path)
      out.write(frame)

  # Cleanup
  out.release()
  cv2.destroyAllWindows()

__all__ = {
  "write_video": write_video
}



python write images to ffmpeg stream?

ffmpeg -i http://192.168.4.22:8080/video -i http://192.168.4.22:8080/audio.wav -c:v libx264 -preset ultrafast -b:v 4500k -maxrate 4500k -bufsize 9000k -g 50 -c:a aac -b:a 128k -ar 44100 -f flv rtmp://a.rtmp.youtube.com/live2/dvqv-7eyf-72d5-vaw7-dvsb



In [ ]:
import cv2
import subprocess
import time
import pyautogui
import numpy as np

# Take a screenshot and save it to a file
screenshot = pyautogui.screenshot()

width, height = screenshot.size

# Video properties
#width, height, fps = 1280, 720, 30
fps = 4

# FFmpeg command for encoding & saving video
ffmpeg_cmd = [
    "ffmpeg",
    "-y",
    "-f", "image2pipe",
    "-vcodec", "mjpeg",
    "-r", str(fps),
    "-i", "-",  # Read from stdin
    "-c:v", "libx264",
    "-preset", "ultrafast",
    "-b:v", "4500k",
    "-maxrate", "4500k",
    "-bufsize", "9000k",
    "-g", "50",
    "-f", "flv",
    "rtmp://a.rtmp.youtube.com/live2/dvqv-7eyf-72d5-vaw7-dvsb"
]

def stream_images(images):

  # Start FFmpeg process
  process = subprocess.Popen(ffmpeg_cmd, stdin=subprocess.PIPE)

  # OpenCV VideoWriter (encoding to a memory buffer)
  # fourcc = cv2.VideoWriter_fourcc(*"H264")
  # video_writer = cv2.VideoWriter(process.stdin, fourcc, fps, (width, height))

  # OpenCV video capture
  #cap = cv2.VideoCapture(0)

  try:
    while True:
      #ret, frame = cap.read()
      #if not ret:
      #    break
      screenshot = pyautogui.screenshot()
      screenshot_np = np.array(screenshot)

      # Convert RGB to BGR (OpenCV uses BGR format)
      screenshot_cv = cv2.cvtColor(screenshot_np, cv2.COLOR_RGB2BGR)

      _, jpeg_bytes = cv2.imencode(".jpg", screenshot_cv, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

      # Write frame to OpenCV VideoWriter
      # video_writer.write(screenshot_cv)

      # Get encoded frame and send to FFmpeg
      process.stdin.write(jpeg_bytes.tobytes())

      time.sleep(1 / fps / 2)

      # Get encoded frame and send to FFmpeg
      process.stdin.write(jpeg_bytes.tobytes())

      time.sleep(1 / fps / 2)

  except KeyboardInterrupt:
    print("\nCTRL+C detected. Exiting...")
  # Cleanup
  # cap.release()
  video_writer.release()
  process.stdin.close()
  process.wait()

__all__ = {
  "stream_images": stream_images
}
